# ETL Australian users reviews

Este notebook presenta la extracción, transformación y carga de el data frame de reseñas de usuarios Australianos de Steam, junto con los criterios de por que se modifico, conservó o eliminó cada parte de la información.

---

# 1.Importamos las librerías que usaremos.

In [1]:
import nltk # Con nltk procesaremos el lenguaje natural para convertirlo a una puntuación que nos dirá si la reseña es negativa, positiva o neutra.
from nltk.sentiment import SentimentIntensityAnalyzer # Usaremos el analizador de sentimientos de nltk.
from langdetect import detect # Usaremos la función detect de langdetect para detectar el idioma en el que está escrita la reseña.
import pandas as pd # Nuestra librearía para trabajar datos.
import re # Expresiones regulares que nos serán útiles para buscar texto en las reseñas.

# Funcionalidades de nltk para tratar el lenguaje natural que usaremos en las reseñas.
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Descargamos las funciones de nltk.
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('vader_lexicon')

C:\Users\USUARIO\AppData\Local\Temp\ipykernel_42644\2909002933.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd # Nuestra librearía para trabajar datos.
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\USUARIO\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\USUARIO\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\USUARIO\AppData\Roaming\nltk_data...
[nltk_data]   Package wordne

True

# 2. Carga de datos.

Debemos cargar el dataframe de una manera distinta, ya que está mal formateado el JSON, para ello cree una función que dada una ruta hacia un json, realiza un proceso distinto al de carga de pd.read_json() para lograr cargar el dataframe.

In [37]:
def json_problematico(path):
    '''
    Crea un dataframe cuando el método pd.read_json() falla debido a que el JSON origen tiene problemas de sintaxis como usar comillas distintas entre otras.

    requiere:
    pandas y ast.
    recibe:
    Ruta del archivo JSON.
    retorna:
    dataframe de pandas con dicho archivo JSON.
    '''
    # Importamos las librerías necesarias.
    import ast
    import pandas as pd
    # Una lista para guardar los resultados y luego convertirla a DataFrame.
    datos = []
    # Abrimos el archivo
    with open(path, 'r', encoding='utf-8') as f:
        for i, line in enumerate(f):
            try:
                # Convertimos cada linea a un diccionario de python.
                json_line = ast.literal_eval(line)
                # Agregamos la linea a los datos.
                datos.append(json_line)
            # Tratamos los errores con un output en la linea que se produjo
            except:
                print(f"Error en la línea: {i+1}")
                continue
    # Creamos el data frame con los datos resultantes y lo retornamos.
    df = pd.DataFrame(datos)
    return df

---

cargamos el JSON con la función y empezamos a realizar las primeras revisiones a este.

In [38]:
df_rev = json_problematico('../Data/australian_user_reviews.json') # Carga

# 3. Primera vista y limpieza.

In [39]:
df_rev.head() # Preview

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


Ya que los datos de reviews están en lista, necesitamos desempaquetarlos para poder usarlos en columnas y realizar una limpieza y el EDA a futuro correctamente. Tampoco usaremos la columna url de los usuarios asi que la borramos directamente.

In [40]:
df_rev = df_rev.drop(columns=['user_url']).reset_index(drop=True)

Primero que todo, eliminamos los duplicados ya que esto puede hacer que el proceso se tarde aún más y llenar el nuevo data frame con información repetida exponencialmente. También eliminamos los valores faltantes para no hacer iteraciones innecesarias.

In [41]:
df_rev[df_rev.duplicated(subset=['user_id'])] # Primero observamos cuantos hay.

,user_id,reviews
456,bokkkbokkk,"[{'funny': '', 'posted': 'Posted September 24,..."
1182,ImSeriouss,"[{'funny': '', 'posted': 'Posted January 10, 2..."
1456,76561198062039159,"[{'funny': '', 'posted': 'Posted August 24, 20..."
1477,76561198045009232,"[{'funny': '', 'posted': 'Posted October 31, 2..."
1746,nitr0ticwolf,"[{'funny': '', 'posted': 'Posted December 12, ..."
...,...,...
17819,76561198076474887,"[{'funny': '', 'posted': 'Posted April 12.', '..."
17916,yolofaceguy,"[{'funny': '', 'posted': 'Posted October 31, 2..."
18028,76561198075591109,"[{'funny': '', 'posted': 'Posted December 26, ..."
18234,76561198092022514,"[{'funny': '', 'posted': 'Posted July 3.', 'la..."


In [42]:
df_rev = df_rev.drop_duplicates(subset=['user_id']) # Borramos los duplicados.

In [43]:
df_rev.isna().sum() # Seguimos con los faltantes.

user_id    0
reviews    0
dtype: int64

# 4. Transformación.

Al no haber valores faltantes, empezamos con nuestro objetivo de crear el nuevo dataframe con cada review y sus características.

En este caso no me llevaré las columnas funny y helpful, ya que no las considero útiles ni para la API ni para el algoritmo de recomendación.

In [44]:
keys = ['posted', 'item_id', 'recommend', 'review'] # Elegimos las columnas que queremos que tenga nuestro nuevo dataframe.
df = pd.DataFrame(columns=['id', *keys]) # Creamos el data frame con las columnas id y las llaves definirán el resto de columnas.
# Iteramos en las reviews, y lo indexamos para poder acceder al id.
for i, revs in enumerate(df_rev['reviews']):
    id = df_rev['user_id'].iloc[i] # Extraemos el id con el indice.
    # Iteramos en las reviews de el usuario.
    for rev in revs:
        # Por cada review, se crea una fila con el id previamente extraído, más los datos en cada una de las llaves que previamente extrajimos.
        row = [id, *[rev[key] for key in keys]]
        # Agregamos la fila al dataframe nuevo.
        df.loc[len(df)] = row

Evaluemos el dataframe resultante y si es correcto podemos eliminar el anterior para ahorrar recursos.

In [45]:
df.head()

,id,posted,item_id,recommend,review
0,76561197970982479,"Posted November 5, 2011.",1250,True,Simple yet with great replayability. In my opi...
1,76561197970982479,"Posted July 15, 2011.",22200,True,It's unique and worth a playthrough.
2,76561197970982479,"Posted April 21, 2011.",43110,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,"Posted June 24, 2014.",251610,True,I know what you think when you see this title ...
4,js41637,"Posted September 8, 2013.",227300,True,For a simple (it's actually not all that simpl...


Por encima se ve correcto, pero me gustaría probarlo.

In [46]:
# Iteramos en el ID y la cantidad de veces que aparece cada ID en nuestro nuevo dataframe.
for id, cant in df['id'].value_counts().items():
    # Las veces que se espera que aparezca ese ID son la cantidad de reviews que tiene ese usuario en nuestro antiguo data frame, las extraemos sabiendo la longitud de la lista que arrojen dichas reviews.
    exp = len(df_rev[df_rev['user_id'] == id]['reviews'].iloc[0])
    # Si la cantidad de veces que aparece el usuario es diferente a la esperada, nos dice qué usuario está causándolo y detiene el programa.
    if cant != exp:
        print(f'Hay {cant} reviews, se esperaban {exp}. Id: {id}')
        break

Al no haberse detenido el programa con el output, sabemos que el dataframe resultante está bien, procedemos a borrar el anterior y seguir con nuestro objetivo.

In [47]:
del df_rev

---

# 5. Segunda transformación ( Análisis de sentimientos )

### 5.1 Limpieza

Empecemos limpiando el apartado que usaremos, que es el de review, ya que pueden haber caracteres o texto que haga más lento y menos exacto el proceso de evaluación de sentimientos.

In [48]:
def clean_text(text):
    '''
    Limpia texto.

    Elimina los caracteres que no sean alfanuméricos del texto y vuelve minúsculas las letras
    
    Requiere:
    importar re (regular expressions).

    Recibe:
    Un String.

    Retorna:
    El string optimizado para su procesamiento como lenguaje natural.
    '''
    # Quitamos los caracteres que no sean letras o números.
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text) 
    # Convertimos el texto a minúsculas.
    text = text.lower()
    # Tokenizamos el texto.
    tokens = word_tokenize(text)
    # Eliminamos las stopworlds.
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    # Realizamos la Lematización.
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    # Unimos los tokens en texto limpio y lo retornamos.
    clean_text = ' '.join(tokens)
    return clean_text

In [49]:
df['review'] = df['review'].apply(clean_text)

### 5.2 Transformación.

Con el texto limpio, procedemos a identificar el idioma en el que esté escrito cada reseña.

In [50]:
def detect_language(text):
    '''
    Detecta idioma.

    Detecta el idioma de un String.

    Requiere:
    importar detect ( Ver comienzo del notebook ).

    Recibe:
    Un String.

    Retorna:
    Idioma del texto simplificado, por ejemplo inglés: "en", español: "es", etc... Si no detecta texto, retorna None.
    '''
    # Intenta detectar el idioma, si no hay texto retorna None
    try:
        return detect(text)
    except:
        return None

df['lang'] = df['review'].apply(detect_language)

Observemos qué idiomas detectó nuestro script.

In [51]:
df['lang'].value_counts()

lang
en    38570
pt     2117
so     1831
da     1679
es     1675
af     1551
no      969
cy      952
nl      832
tl      783
it      633
fr      519
ro      512
sl      446
ca      437
pl      419
et      337
id      320
tr      310
sw      305
hr      260
sk      214
fi      201
sv      182
de      166
sq      147
hu      127
lt       98
cs       54
lv       49
vi       36
Name: count, dtype: int64

Ya que hay muchos idiomas, pero en su gran mayoría las reseñas están en inglés, mapearé el top 5 de los idiomas para un futuro EDA y a los que queden fuera los categorizaremos como otros.

In [52]:
# Creamos un mapeo para centrarnos en el top 5 de idiomas en las reseñas.
def map_lang(lang):
    maps = {
        "en": "Inglés",
        "pt": "Portugués",
        "es": "Español",
        "so": "Somalí",
        "da": "Danés"
    }
    return maps.get(lang, "Otro")
# Aplicamos la función al dataframe.
df['lang'] = df['lang'].apply(map_lang)

In [53]:
df['lang'].value_counts()

lang
Inglés       38570
Otro         12558
Portugués     2117
Somalí        1831
Danés         1679
Español       1675
Name: count, dtype: int64

Por los siguientes motivos:


1. Cantidad de registros en inglés ( No perderemos muchos datos ).
2. Optimización y entrenamiento de los algoritmos de lenguaje natural o análisis de sentimiento para el lenguaje Inglés.
3. Rendimiento de la transformación de los datos ( menos registros, menos trabajo computacional ).
4. Previo intento de traducción con la API de google translator en una función usando pandas ( Intento fallido debido a la cantidad de registros ).
5. Por último, si el script de idiomas pudo identificar el inglés, es porque hay un texto legible que puede facilitar la detección de sentimiento.

He optado por únicamente utilizar únicamente las reseñas en inglés para el analizador de sentimientos, sin embargo, los demás idiomas nos pueden ser útiles en el EDA para ver la diversidad de idiomas que tienen los usuarios de Steam ubicados en Australia.

Empezamos guardando los idiomas para su futuro análisis.

In [54]:
# El total de registros para luego calcular el porcentaje.
total = len(df)
# Diccionario para luego crear el dataframe.
langs = {}
# Iteramos en las veces que aparecen los idiomas en nuestro data frame, junto con el idioma, a esta tupla la llamamos elm.
for elm in df['lang'].value_counts().items():
    # El porcentaje será la cantidad de apariciones sobre el total.
    perc = elm[1]/total
    # Formateamos el porcentaje para una mejor vista.
    perc = str(perc*100)[:4] + ' %'
    # Lo agregamos al diccionario con la llave del nombre del idioma.
    langs[elm[0]] = [elm[1], perc]
# Creamos el data frame y lo guardamos como CSV para luego usarlo
df_langs = pd.DataFrame.from_dict(langs, orient='index', columns=['Cantidad', 'Porcentaje'])
df_langs.to_csv('../Data/Processed/langs_EDA.csv')
display(df_langs) # Revisamos el dataframe
# Borramos lo que no usaremos.
del df_langs, langs

,Cantidad,Porcentaje
Inglés,38570,66.0 %
Otro,12558,21.4 %
Portugués,2117,3.62 %
Somalí,1831,3.13 %
Danés,1679,2.87 %
Español,1675,2.86 %


Ahora filtramos el df para que solo tenga las reviews en inglés.

In [55]:
df = df[df['lang'] == 'Inglés']

### 5.3 Función sentimientos.

Para hacer el análisis de sentimiento cree la función sentimiento que se encargará de recibir el texto, y retornar lo que el algoritmo considere. Consultar la documentación de la función.

In [56]:
def sentimiento(texto, sid=None):
    '''
    Analizador de sentimiento.

    Detecta el sentimiento de un texto y retorna un número en base a el puntaje obtenido.
    
    Requiere:
    nltk y su paquete de descarga ( Consultar comienzo del notebook )

    Recibe:
    Un string y un objeto de SentimentIntensityAnalyzer() (opcional pero recomendado), Si no existe lo crea.

    Retorna:
    0 para sentimientos negativos, 1 para sentimientos neutrales (O las que no le encuentre sentido) y 2 para sentimientos positivos.
    '''
    # Creamos el analizador de sentimiento si no existe.
    if not sid:
        sid = SentimentIntensityAnalyzer()
    # Obtenemos el puntaje de sentimiento.
    scores = sid.polarity_scores(texto)
    # Interpretamos el puntaje obtenido y retornamos el número esperado.
    if scores['compound'] >= 0.05:
        return 2
    elif scores['compound'] <= -0.05:
        return 0
    else:
        return 1

Ahora creamos el analizador de sentimientos y lo usamos en nuestro dataframe para crear la nueva columna.

In [57]:
sid = SentimentIntensityAnalyzer()
df['sentiment'] = df['review'].apply(sentimiento, args=[sid])

Eliminamos la columna reseñas ya que ya no la usaremos más.

In [58]:
df = df.drop(columns=['review']).reset_index(drop=True)

# 6. Última transformación ( Columna Fecha ).

Por último transformamos la columna posted a su año para el EDA de este dataframe, información la cual nos será útil.

In [59]:
# Extraemos el año con una función.
def years(texto):
    # Expresión regular para encontrar el año (4 dígitos)
    patron = r'(\d{4})'  
    resultado = re.findall(patron, texto)
    if resultado:
        return resultado[0]  # Devuelve el primer año encontrado
    else:
        return None  # Si no se encuentra el año, devuelve None

# Aplicar la función a cada fila del DataFrame
df['year'] = df['posted'].apply(years)

Verificamos que quedó bien.

In [60]:
df['year'].unique()

array(['2011', '2014', '2013', None, '2012', '2015', '2010'], dtype=object)

In [61]:
df.head()

,id,posted,item_id,recommend,lang,sentiment,year
0,76561197970982479,"Posted November 5, 2011.",1250,True,Inglés,2,2011
1,76561197970982479,"Posted July 15, 2011.",22200,True,Inglés,2,2011
2,76561197970982479,"Posted April 21, 2011.",43110,True,Inglés,2,2011
3,js41637,"Posted June 24, 2014.",251610,True,Inglés,2,2014
4,js41637,"Posted September 8, 2013.",227300,True,Inglés,2,2013


Por último eliminamos las columnas restantes lang y posted.

In [62]:
df = df.drop(columns=['posted', 'lang'])

In [63]:
df.head()

,id,item_id,recommend,sentiment,year
0,76561197970982479,1250,True,2,2011
1,76561197970982479,22200,True,2,2011
2,76561197970982479,43110,True,2,2011
3,js41637,251610,True,2,2014
4,js41637,227300,True,2,2013


# 7. Carga

Finalmente cargamos los datos. En este caso, el dataframe resultante será optimo tanto para la API, como para el EDA y el algoritmo de recomendación. Conservaré los años nulos en ambas ya que en el EDA solo se usarán para describir el crecimiento de las reviews y en el API solo se usaran en una función.

In [64]:
df.to_csv('../Data/Processed/df_reviews.csv', index=False)